In [ ]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 

morph = MorphAnalyzer()

## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

## Данные

In [ ]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def clean(words):
    clean = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return clean

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [ ]:
texts = open('habr_texts.txt').read().splitlines()

In [ ]:
f = open('habr_texts.vw', 'w')

for i, text in enumerate(habr_texts):
    c = Counter(text)
    doc = 'doc_'+ str(i) + ' '
    vw_text = ' '.join([x+':'+str(c[x]) for x in c])
    
    f.write(doc + vw_text  + '\n')
f.close()



В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [ ]:
texts = [clean(word_tokenize(text.lower())) for text in texts]

In [ ]:
# для нграммов
# ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
# p = gensim.models.phrases.Phraser(ph)
# ngrammed_texts = p[texts]

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [ ]:
dictinary = gensim.corpora.Dictionary(texts)

In [ ]:
dictinary.filter_extremes(no_above=0.3, no_below=30)
dictinary.compactify()

In [ ]:
print(dictinary)

Преобразуем наши тексты в мешки слов. 

In [ ]:
corpus = [dictinary.doc2bow(text) for text in texts]
# если текстов много, то тут может быть генератор

In [ ]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, passes=1) # если поддерживается многопоточность
# lsi = gensim.models.LdaModel(200, id2word=dictinary, passes=5)

Посмотрим на топики.

In [ ]:
lda.print_topics()

Ещё есть штука для визуализации.

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(lda, corpus, dictinary)

Можно посмотреть метрики.

In [ ]:
import numpy as np

In [ ]:
?lda.log_perplexity

In [ ]:
lda.log_perplexity(corpus[:2000], total_docs=100)

In [ ]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [ ]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
coherence_model_lda.get_coherence()

### Разложение матриц в sklearn

In [ ]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [ ]:
stexts = [' '.join(text) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

Разложим её.

In [ ]:
model = NMF(n_components=30)

In [ ]:
model.fit(X)

In [ ]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [ ]:
get_nmf_topics(model, 10)